# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [20]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("~\OneDrive\Desktop\Bootcamp\Module6\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,11.07,81,99,5.79,overcast clouds
1,1,Yinchuan,CN,38.4681,106.2731,20.98,32,4,5.88,clear sky
2,2,Hermanus,ZA,-34.4187,19.2345,66.79,95,64,5.61,light rain
3,3,Vaini,TO,-21.2000,-175.2000,77.16,94,100,13.80,moderate rain
4,4,Rikitea,PF,-23.1203,-134.9692,75.49,79,5,21.88,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [21]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [22]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hermanus,ZA,-34.4187,19.2345,66.79,95,64,5.61,light rain
3,3,Vaini,TO,-21.2000,-175.2000,77.16,94,100,13.80,moderate rain
4,4,Rikitea,PF,-23.1203,-134.9692,75.49,79,5,21.88,clear sky
5,5,Hambantota,LK,6.1241,81.1185,70.14,100,20,3.44,few clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,72.99,85,20,9.42,light rain
7,7,La Virginia,CO,4.8997,-75.8825,71.15,82,20,5.75,few clouds
8,8,Quelimane,MZ,-17.8786,36.8883,76.95,94,20,2.30,few clouds
10,10,Dillon,US,34.4166,-79.3712,50.25,64,0,8.75,clear sky
11,11,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,83,75,4.61,broken clouds
16,16,Punta Arenas,CL,-53.1500,-70.9167,51.91,71,40,23.02,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [23]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hermanus,ZA,-34.4187,19.2345,66.79,95,64,5.61,light rain
3,3,Vaini,TO,-21.2000,-175.2000,77.16,94,100,13.80,moderate rain
4,4,Rikitea,PF,-23.1203,-134.9692,75.49,79,5,21.88,clear sky
5,5,Hambantota,LK,6.1241,81.1185,70.14,100,20,3.44,few clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,72.99,85,20,9.42,light rain
7,7,La Virginia,CO,4.8997,-75.8825,71.15,82,20,5.75,few clouds
8,8,Quelimane,MZ,-17.8786,36.8883,76.95,94,20,2.30,few clouds
10,10,Dillon,US,34.4166,-79.3712,50.25,64,0,8.75,clear sky
11,11,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,83,75,4.61,broken clouds
16,16,Punta Arenas,CL,-53.1500,-70.9167,51.91,71,40,23.02,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [24]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [25]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,66.79,light rain,-34.4187,19.2345,
3,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
4,Rikitea,PF,75.49,clear sky,-23.1203,-134.9692,
5,Hambantota,LK,70.14,few clouds,6.1241,81.1185,
6,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,
7,La Virginia,CO,71.15,few clouds,4.8997,-75.8825,
8,Quelimane,MZ,76.95,few clouds,-17.8786,36.8883,
10,Dillon,US,50.25,clear sky,34.4166,-79.3712,
11,Arraial Do Cabo,BR,78.76,broken clouds,-22.9661,-42.0278,
16,Punta Arenas,CL,51.91,scattered clouds,-53.1500,-70.9167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [26]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [27]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Hermanus - nearest hotel: Aloe guest house
Vaini - nearest hotel: Keleti International Resort
Rikitea - nearest hotel: Chez Bianca & Benoit
Hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
Saint-Philippe - nearest hotel: Le Baril
La Virginia - nearest hotel: Restaurante y Hospedaje Matejal
Quelimane - nearest hotel: Mabassa
Dillon - nearest hotel: Quality Inn
Arraial Do Cabo - nearest hotel: No hotel found
Punta Arenas - nearest hotel: Hotel Lacolet
Inirida - nearest hotel: Residencias El Turista del Guainia
Meulaboh - nearest hotel: Hotel Meuligou
Cape Town - nearest hotel: Townhouse Hotel
Padang - nearest hotel: losmen Surya
Victoria - nearest hotel: 四季酒店 Four Seasons Hotel
Avarua - nearest hotel: Paradise Inn
Namibe - nearest hotel: Hotel Chik Chik Namibe
Butaritari - nearest hotel: No hotel found
Busselton - nearest hotel: No hotel found
Wokha - nearest hotel: No hotel found
Arlit - nearest hotel: No hotel found
Caravelas - nearest hotel: Pousada do

Vao - nearest hotel: Nataiwatch
Isangel - nearest hotel: Tanna Lodge
Mahebourg - nearest hotel: Grand Bel Air
Takoradi - nearest hotel: Hillcrest
San Pedro - nearest hotel: Hotel Takara
Praia - nearest hotel: Casa Sodadi
Sabang - nearest hotel: Sabang Hill
Lourdes - nearest hotel: Florence Hôtel
Soyo - nearest hotel: Hotel Maria Teres
Guanay - nearest hotel: Ritzi
Belmonte - nearest hotel: No hotel found
Tshikapa - nearest hotel: Hôtel TSHAMUNTU
Trelew - nearest hotel: Patagonia Suites & Apart
Kismayo - nearest hotel: Kismayo Hotel
Japura - nearest hotel: No hotel found
Gat - nearest hotel: No hotel found
Oranjestad - nearest hotel: Hotel California
Aitape - nearest hotel: No hotel found
Sabha - nearest hotel: فندق الكاظم-سبها
Sao Paulo De Olivenca - nearest hotel: No hotel found
Kununurra - nearest hotel: Kununurra Country Club Resort
Lakes Entrance - nearest hotel: Banjo Patterson Motor Inn
Kulhudhuffushi - nearest hotel: No hotel found
Pisco - nearest hotel: La Portada
Saldanha - ne

Ternate - nearest hotel: Boulivard ternate
Cabra - nearest hotel: Pensión Guerrero
Ewo - nearest hotel: No hotel found
Gasa - nearest hotel: مسجد الفتح المبين
Labuan - nearest hotel: Labuan Avenue Hotel
Karaton - nearest hotel: No hotel found
Athens - nearest hotel: Eridanus
The Valley - nearest hotel: CeBlue
Aojiang - nearest hotel: No hotel found
Gonaives - nearest hotel: amiral kilik
Kumphawapi - nearest hotel: กุมภวาปีรีสอร์ท


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,66.79,light rain,-34.4187,19.2345,Aloe guest house
3,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti International Resort
4,Rikitea,PF,75.49,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
5,Hambantota,LK,70.14,few clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
6,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,Le Baril
...,...,...,...,...,...,...,...
678,Athens,GR,61.18,scattered clouds,37.9795,23.7162,Eridanus
679,The Valley,AI,77.09,clear sky,18.2170,-63.0578,CeBlue
682,Aojiang,CN,59.38,broken clouds,22.9421,116.0359,No hotel found
685,Gonaives,HT,72.66,clear sky,19.4500,-72.6833,amiral kilik


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [28]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,66.79,light rain,-34.4187,19.2345,Aloe guest house
3,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti International Resort
4,Rikitea,PF,75.49,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
5,Hambantota,LK,70.14,few clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
6,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,Le Baril
7,La Virginia,CO,71.15,few clouds,4.8997,-75.8825,Restaurante y Hospedaje Matejal
8,Quelimane,MZ,76.95,few clouds,-17.8786,36.8883,Mabassa
10,Dillon,US,50.25,clear sky,34.4166,-79.3712,Quality Inn
11,Arraial Do Cabo,BR,78.76,broken clouds,-22.9661,-42.0278,No hotel found
16,Punta Arenas,CL,51.91,scattered clouds,-53.1500,-70.9167,Hotel Lacolet


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [29]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [30]:
#configure hotel map
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1.0,
    color = "City",
    hover_cols = ["City","Hotel Name", "Country","Current Description"]
)

hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)